In [3]:
import os
import numpy as np
import pandas as pd
import lightgbm as lgb
from datetime import datetime

from config.config import (
    FE_PATH,
    MES_TEST_FINAL,
    MODELOS_PATH,
    RESULTADOS_PREDICCION_PATH,
    NOMBRE_EXPERIMENTO,
)
from src.data_load_preparation import cargar_datos, preparar_test_final


In [4]:
# Cargar dataset de features
data = cargar_datos(FE_PATH)

# Test final según config (MES_TEST_FINAL = [202108])
X_test, clientes_test = preparar_test_final(data)

# Por si querés también el subset completo de 202108 con todas las columnas:
data_test = data[data["foto_mes"].isin(MES_TEST_FINAL)].copy()

len(X_test), len(clientes_test), len(data_test)


2025-11-16 22:20:16,607 - INFO - 📥 Cargando dataset desde: /home/sanmartinofacundo/buckets/b1/features/fe_v3/competencia_02_fe_v3.parquet


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

2025-11-16 22:28:58,402 - INFO - ✅ Dataset cargado con 4,717,958 filas y 673 columnas
2025-11-16 22:29:26,699 - INFO - 📊 Test final: 164,822 registros de meses [202108]
2025-11-16 22:29:27,076 - INFO - ✅ X_test: (164822, 672)


(164822, 164822, 164822)

In [5]:
import glob

model_pattern = os.path.join(MODELOS_PATH, f"{NOMBRE_EXPERIMENTO}_seed*_final.txt")
model_files = sorted(glob.glob(model_pattern))

print("Modelos encontrados:")
for f in model_files:
    print(" -", os.path.basename(f))


Modelos encontrados:
 - lgbm_fe_v3_us010_tr201901-202102_val202104_vext202106_test202108_s7_seed100003_final.txt
 - lgbm_fe_v3_us010_tr201901-202102_val202104_vext202106_test202108_s7_seed100019_final.txt
 - lgbm_fe_v3_us010_tr201901-202102_val202104_vext202106_test202108_s7_seed100043_final.txt
 - lgbm_fe_v3_us010_tr201901-202102_val202104_vext202106_test202108_s7_seed100049_final.txt
 - lgbm_fe_v3_us010_tr201901-202102_val202104_vext202106_test202108_s7_seed100057_final.txt
 - lgbm_fe_v3_us010_tr201901-202102_val202104_vext202106_test202108_s7_seed182009_final.txt
 - lgbm_fe_v3_us010_tr201901-202102_val202104_vext202106_test202108_s7_seed182011_final.txt
 - lgbm_fe_v3_us010_tr201901-202102_val202104_vext202106_test202108_s7_seed182027_final.txt
 - lgbm_fe_v3_us010_tr201901-202102_val202104_vext202106_test202108_s7_seed200003_final.txt
 - lgbm_fe_v3_us010_tr201901-202102_val202104_vext202106_test202108_s7_seed200017_final.txt
 - lgbm_fe_v3_us010_tr201901-202102_val202104_vext202106_te

In [6]:
all_preds = []

for model_path in model_files:
    print(f"📂 Cargando modelo: {os.path.basename(model_path)}")
    model = lgb.Booster(model_file=model_path)
    y_pred = model.predict(X_test)
    all_preds.append(y_pred)

all_preds = np.vstack(all_preds)   # shape = (n_modelos, n_clientes)
prob_ensemble = all_preds.mean(axis=0)

prob_ensemble.shape


📂 Cargando modelo: lgbm_fe_v3_us010_tr201901-202102_val202104_vext202106_test202108_s7_seed100003_final.txt
📂 Cargando modelo: lgbm_fe_v3_us010_tr201901-202102_val202104_vext202106_test202108_s7_seed100019_final.txt
📂 Cargando modelo: lgbm_fe_v3_us010_tr201901-202102_val202104_vext202106_test202108_s7_seed100043_final.txt
📂 Cargando modelo: lgbm_fe_v3_us010_tr201901-202102_val202104_vext202106_test202108_s7_seed100049_final.txt
📂 Cargando modelo: lgbm_fe_v3_us010_tr201901-202102_val202104_vext202106_test202108_s7_seed100057_final.txt
📂 Cargando modelo: lgbm_fe_v3_us010_tr201901-202102_val202104_vext202106_test202108_s7_seed182009_final.txt
📂 Cargando modelo: lgbm_fe_v3_us010_tr201901-202102_val202104_vext202106_test202108_s7_seed182011_final.txt
📂 Cargando modelo: lgbm_fe_v3_us010_tr201901-202102_val202104_vext202106_test202108_s7_seed182027_final.txt
📂 Cargando modelo: lgbm_fe_v3_us010_tr201901-202102_val202104_vext202106_test202108_s7_seed200003_final.txt
📂 Cargando modelo: lgbm_fe_v

(164822,)

In [7]:
N_optimo = 10500  # por ejemplo

# Ordenar prob de mayor a menor
indices_ordenados = np.argsort(prob_ensemble)[::-1]

# Seleccionar top N
topN_idx = indices_ordenados[:N_optimo]

# Obtenemos sólo los clientes a incentivar
clientes_incentivar = clientes_test[topN_idx]


In [8]:
os.makedirs(RESULTADOS_PREDICCION_PATH, exist_ok=True)

timestamp = datetime.now().strftime("%Y%m%d_%H%M")
filename = (
    f"{NOMBRE_EXPERIMENTO}_topN_{N_optimo}_{timestamp}.csv"
)

output_path = os.path.join(RESULTADOS_PREDICCION_PATH, filename)

# Guardar CSV sin header ni índice
pd.DataFrame(clientes_incentivar).to_csv(
    output_path,
    index=False,
    header=False
)

output_path

'/home/sanmartinofacundo/buckets/b1/competencia_02/lgbm_fe_v3_us010_tr201901-202102_val202104_vext202106_test202108_s7/resultados_prediccion/lgbm_fe_v3_us010_tr201901-202102_val202104_vext202106_test202108_s7_topN_10500_20251116_2231.csv'